# TP4 - Non-negative Matrix Factorization
The goal is to study the use of nonnegative matrix factorisation (NMF) for topic extraction from a dataset of text documents. The rationale is to interpret each extracted NMF component as being associated with a specific topic. 

Study and test the following script (introduced  on [scikit](http://scikit-learn.org/stable/auto_examples/applications/plot_topics_extraction_with_nmf_lda.html))

In [1]:
from time import time

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import NMF, LatentDirichletAllocation
from sklearn.datasets import fetch_20newsgroups

In [2]:
def vectorizeFeatures(_vectorizer=None, _random_state=None):
    # Set default params
    if _vectorizer is None:
        vectorizer = TfidfVectorizer(max_df=0.95, min_df=2, max_features=1000, stop_words='english')
    else:
        vectorizer = _vectorizer
    random_state = 1 if _random_state is None else _random_state
    # Fetch data and vectorize
    print("Loading dataset...")
    dataset = fetch_20newsgroups(shuffle=True, random_state=random_state,
                                 remove=('headers', 'footers', 'quotes'))
    data_samples = dataset.data[:2000]        
    t0 = time()
    features = vectorizer.fit_transform(data_samples)
    feature_names = vectorizer.get_feature_names()
    print("done in %0.3fs." % (time() - t0))
    return features, feature_names

In [3]:
def NMFModel(features, _vectorizerName=None, _random_state=None, 
             _beta_loss=None, _init=None, _W=None, _H=None, _K = None):
    
    n_samples = 2000
    n_features = 1000
    n_top_words = 20
    n_components = 10 if _K is None else _K
    vectorizerName = "tf_idf" if _vectorizerName is None else _vectorizerName
    random_state = 1 if _random_state is None else _random_state
    solver = 'cd' if _beta_loss is None else 'mu'
    beta_loss = 'frobenius' if _beta_loss is None else _beta_loss
    init = 'random' if _init is None else _init
    
    print("Fitting the NMF model ("+beta_loss+" norm) with "+vectorizerName+" features, "
          "n_samples=%d and n_features=%d..." % (n_samples, n_features))
    
    t0 = time()
    if _init is None:
        nmf = NMF(n_components=n_components, 
                  random_state=_random_state,
                  solver = solver,
                  beta_loss = beta_loss,
                  init = 'random',
                  alpha=.1, l1_ratio=.5).fit(features)
    else:
        nmf = NMF(n_components=n_components, 
                  random_state=_random_state,
                  solver = solver,
                  beta_loss = beta_loss,
                  init = _init,
                  alpha=.1, l1_ratio=.5)
        nmf.fit_transform(features, W=_W, H=_H)
    print("done in %0.3fs." % (time() - t0))

    print("\nTopics in NMF model ("+beta_loss+" norm):")
    return nmf, n_top_words

In [4]:
def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        message = "Topic #%d: " % topic_idx
        message += " ".join([feature_names[i]
                             for i in topic.argsort()[:-n_top_words - 1:-1]])
        print(message)
    print()

In [5]:
def runExample(_vectorizer=None, _vectorizerName=None, _random_state=None, _beta_loss=None, 
               _init=None, _W=None, _H=None, _K=None):
    features, feature_names = vectorizeFeatures(_vectorizer, _random_state)
    nmf, n_top_words = NMFModel(features, _vectorizerName, _random_state, _beta_loss, _init, _W, _H, _K)
    print_top_words(nmf, feature_names, n_top_words)

### Q1. Test and comment on the effect of varying the initialisation, especially using random nonnegative values as initial guesses (for W and H coefficients, using the notations introduced during the lecture).

< to be rephrased >

Three different initialisation configuration were tested, while mantaining the same cost function (frobenius norm) and using multiplicative update rules. The 'nndsvda' initializiation, as from the scikit documentation, performs a Nonnegative Double Singular Value Decomposition on the features and then fills zero values with the average value of the features matrix. The 'nndsvdar' initializiation performs the same operation, but fills zeros with very small random values. Finally, the random initialization creates two non-negative random matrices properly scaled.

Of all the three approaches, the one that showed the best results was undoubtly the 'nndsvda', which was able to converge with only 30 iterations and provided the lowest error.

In [6]:
runExample(_init='random')

Loading dataset...
done in 2.695s.
Fitting the NMF model (frobenius norm) with tf_idf features, n_samples=2000 and n_features=1000...
done in 0.549s.

Topics in NMF model (frobenius norm):
Topic #0: just people don like god good know time way say really make ve did ll want new right years year
Topic #1: chip clipper going encryption yes clinton phone used enforcement serial encrypted knows block wonder email doesn market number follow standard
Topic #2: law enforcement right gun crime weapons laws government away citizens people religious jesus block rights federal country fact police control
Topic #3: think don win people extra early sold need toronto means sex actually just pretty happen david wasn mike use list
Topic #4: windows file use dos drive using problem files program software pc window help card running os drivers disk version available
Topic #5: 00 sale card condition 10 price offer 250 asking today new cd tape 50 15 equipment 20 contact 12 software
Topic #6: thanks know do

In [9]:
runExample(_init='nndsvda')

Loading dataset...
done in 1.168s.
Fitting the NMF model (frobenius norm) with tf_idf features, n_samples=2000 and n_features=1000...
done in 0.702s.

Topics in NMF model (frobenius norm):
Topic #0: just people don think like know time good make way really say right ve want did ll new use years
Topic #1: windows use dos using window program os drivers application help software pc running ms screen files version card code work
Topic #2: god jesus bible faith christian christ christians does heaven sin believe lord life church mary atheism belief human love religion
Topic #3: thanks know does mail advance hi info interested email anybody looking card help like appreciated information send list video need
Topic #4: car cars 00 tires miles new engine insurance price condition oil power speed good 000 brake year used models bought
Topic #5: edu soon com send university internet mit ftp mail cc pub article information hope program mac email home contact blood
Topic #6: file problem files for

In [10]:
runExample(_init='nndsvdar')

Loading dataset...
done in 1.434s.
Fitting the NMF model (frobenius norm) with tf_idf features, n_samples=2000 and n_features=1000...
done in 0.866s.

Topics in NMF model (frobenius norm):
Topic #0: just people don think like know time good make way really say right ve want did ll new use years
Topic #1: windows use dos using window program os drivers application help software pc running ms screen files version card code work
Topic #2: god jesus bible faith christian christ christians does heaven sin believe lord life church mary atheism belief human love religion
Topic #3: thanks know does mail advance hi info interested email anybody looking card help like appreciated information send list video need
Topic #4: car cars tires 00 miles new engine insurance price condition oil power speed good 000 brake year used models bought
Topic #5: edu soon com send university internet mit ftp mail cc pub article information hope program mac email home contact blood
Topic #6: file problem files for

### Q2. Compare and comment on the difference between the results obtained with $l_2$ cost compared to the generalised Kullback-Liebler cost.

< to be rephrased >

The Kullback-Lieber cost seems to perform worse than the $ l_{2} $ cost: it needs 3.5 times the iterations to converge. Found topics seem to be similar (religion, business, games...), but also from this representation we can see that the l2 cost produces more accurate results: in topics obtained with l2 cost, all words within the same topic seem to be specific to it, whilst in the topics obtained with Kullback-Leibler cost there seems to be less precision. As an example, we can look at topic #2, where both algorithms find the same topic, but the most important words found by using the l2 cost are much more representative than the ones found using the Kullback-Leibler one.

In [11]:
runExample(_beta_loss='kullback-leibler')

Loading dataset...
done in 1.540s.
Fitting the NMF model (kullback-leibler norm) with tf_idf features, n_samples=2000 and n_features=1000...
done in 17.930s.

Topics in NMF model (kullback-leibler norm):
Topic #0: years case idea make guess say government did law think self talk way wrong talking control point deleted wasn short
Topic #1: wrong sale year 20 offer world red sell 10 price 00 11 condition states 12 30 1993 st 13 50
Topic #2: sure ve ll does mean year remember heard seen said bit doesn hear don true really say thing mentioned head
Topic #3: work using help use works need write type good questions try read problem subject appreciated drive open return tried computer
Topic #4: people god question far say time believe true making understand life actually yes today makes sense support religion university man
Topic #5: good time maybe things thing kind usually bad times like didn day way make thinking look took use fact came
Topic #6: windows looking thanks use mail file hi ava

C:\Users\dgallitelli\AppData\Local\conda\conda\envs\SD-TSIA214\lib\site-packages\sklearn\decomposition\nmf.py:1035: ConvergenceWarning: Maximum number of iteration 200 reached. Increase it to improve convergence.
  " improve convergence." % max_iter, ConvergenceWarning)


### Q3. Test and comment on the results obtained using a simpler term-frequency representation as input (as opposed to the TF-IDF representation considered in the code above) when considering the Kullback-Liebler cost.

< to be rephrased >

Two simpler representations were tested, both the simple Term Frequency representation and the simple Count of tokens. However, neither of them seems to produce better results: if compared to the previous Tf-Idf representation, both of them need

In [12]:
_vectorizer = CountVectorizer(max_df=0.95, min_df=2, max_features=1000, stop_words='english')    
runExample(_beta_loss='kullback-leibler', _vectorizer=_vectorizer, _vectorizerName="CountVectorizer")

Loading dataset...
done in 1.234s.
Fitting the NMF model (kullback-leibler norm) with CountVectorizer features, n_samples=2000 and n_features=1000...
done in 11.983s.

Topics in NMF model (kullback-leibler norm):
Topic #0: new car people years 000 year hiv president health cars research number program time aids used insurance cost engine oil
Topic #1: game year team play win points got season games flyers second years time players couple great mark good just point
Topic #2: 10 11 55 15 20 12 25 18 00 17 16 13 21 19 24 14 22 23 93 40
Topic #3: government law use state public encryption person section gun rights clipper used weapons crime people military control security enforcement self
Topic #4: windows thanks using use file help time problem does software know window color new program need running hi work video
Topic #5: god people does jesus did time say israel believe bible think just church world know fact says life point true
Topic #6: edu com mail graphics send ftp pub available 

____________________________________
## Custom NFM Implementation

In [13]:
###### CUSTOM NMF IMPLEMENTATION ######
# Multiplicative Update Rules for NMF #
# estimation with beta divergences    #
import numpy

# TODO: translate slides 59 [beta-divergence] & 47 [error and special cases]

def custom_NMF(V, K, W=None, H=None, steps=50, beta=0, toll=0.1, show_div=False):
    
    F = len(V) #Number of V rows
    N = len(V[0]) #Number of V columns

    if W is None:
        W = numpy.random.rand(F,K)
        
    if H is None:
        H = numpy.random.rand(K,N)
        
    if N != len(H[0]):
        raise ValueError("Size for H[0] is different - found "+str(len(H[0]))+" in place of "+str(N))
    if F != len(W):
        raise ValueError("Size for F is different - found "+str(len(F))+" in place of "+str(N))
        
    #Setup n_iter
    n_iter = 1
    
    # Setup initial error
    init_error = _beta_div(V,W,H,beta,F,N,K)
    if show_div:
        print("Initial error: "+str(init_error))
    error = init_error
    
    for step in range(steps):
    
#         Tests with whole matrix : multiply = O | dot = *
        upd_UP = numpy.dot(W.T, numpy.multiply(pow(numpy.dot(W,H),beta-2), V))
        upd_DOWN = numpy.dot(W.T, pow(numpy.dot(W,H),beta-1))
        upd = upd_UP / upd_DOWN
        H = numpy.multiply(H, upd)
        
        upd_UP = numpy.dot(numpy.multiply(pow(numpy.dot(W,H),beta-2), V),H.T)
        upd_DOWN = numpy.dot(pow(numpy.dot(W,H),beta-1), H.T)
        upd = upd_UP / upd_DOWN
        W = numpy.multiply(W, upd)
        
        if toll > 0:
            new_error = _beta_div(V,W,H,beta,F,N,K)
            if show_div:
                print("Error on iteration "+str(n_iter)+": " +str(new_error))
            # Check if approximation error relative decrease is below the desired threshold
            if ((error - new_error) / init_error) < toll:
                break
            error = new_error
            
        n_iter += 1
            
    return W, H

def _beta_div(V,W,H,beta,F,N,K):
    div = 0
    # Update beta_divergence
    WH = numpy.dot(W, H)
    for i in range(F):
        for j in range(N):
                x = V[i][j] if V[i][j] != 0 else numpy.finfo(numpy.double).tiny
                y = WH[i][j]
                if beta == 1: # generalized Kullback-Leibler divergence. x log(x/y) - x + y
                    div += x*numpy.log(x/y) - x + y
                elif beta == 0: # Itakura-Saito divergence. (x/y) - log(x/y) -1
                    div += (x/y) * numpy.log(x/y) - 1
                else: # Euclidean distance. (1/beta(beta-1))(x^beta + (beta-1)y^beta - beta*x*y^beta-1)
                    div += 1/(beta*(beta-1))*(pow(x,beta) + (beta-1)*pow(y,beta) - beta*x*pow(y,beta-1))
    return div

#######

In [14]:
features, feature_names = vectorizeFeatures()

V = numpy.random.rand(features.shape[0], features.shape[1])
V = numpy.array(V) # Data matrix F x N 
K = 10

W, H = custom_NMF(V, K, beta = 1, toll = 0.001, show_div = True)

Loading dataset...
done in 1.587s.
Initial error: 2675399.9859820143
Error on iteration 1: 198321.5349171468
Error on iteration 2: 197666.45467101078


In [15]:
runExample(_init='custom', _W=W, _H=H, _K=K)

Loading dataset...
done in 1.380s.
Fitting the NMF model (frobenius norm) with tf_idf features, n_samples=2000 and n_features=1000...
done in 0.041s.

Topics in NMF model (frobenius norm):
Topic #0: young encrypted exactly evidence events especially error eric equipment entire engine enforcement energy end encryption email drive effort effective effect
Topic #1: young encrypted exactly evidence events especially error eric equipment entire engine enforcement energy end encryption email drive effort effective effect
Topic #2: young encrypted exactly evidence events especially error eric equipment entire engine enforcement energy end encryption email drive effort effective effect
Topic #3: young encrypted exactly evidence events especially error eric equipment entire engine enforcement energy end encryption email drive effort effective effect
Topic #4: young encrypted exactly evidence events especially error eric equipment entire engine enforcement energy end encryption email drive effor